# Cleaning up data for data visualization and gmap's use
* Dropping unneccessary data
* Adding categorizing job post that are not categorized
* Dropping the category unknown 
* Format date created column to datetime


In [2]:
#imports 
import json
import requests
import pandas as pd
import numpy as np
import matplotlib as plt
import pprint as pprint

### CSV file path variables

In [3]:
# Raw geo data file to be saved and pulled from
rawGeoData = "../data/rawData/master_geo_raw_dataframe.csv"

# cleaned data file to be saved
cleanData = "../data/cleanData/master_clean_data.csv"

# path csv file location
pathRawData = "../data/rawData/"
pathCleanData ="../datacleanData/"

#### Calling csv file

In [4]:
#calling the csv raw data
geo_csv_file = pd.read_csv(rawGeoData,index_col=0)
geo_csv_file.head(3)

,job title,category,company name,location,date created,lat,lng,latlng,geocode_data,city,state,country
job posting id,,,,,,,,,,,,
1312805843,Physician: Core Faculty Family Medicine Job in...,Healthcare & Nursing Jobs,CompHealth,"Indiana, Indiana County",2019-10-25T12:38:55Z,40.621455,-79.152535,"40.621455,-79.152535","{'address_components': [{'long_name': '1880', ...",Indiana,Pennsylvania,United States
1312804908,Physician: Associate Regional Medical Officer ...,Healthcare & Nursing Jobs,CompHealth,"San Jose, Santa Clara County",2019-10-25T12:38:30Z,37.390494,-121.885434,"37.390494,-121.885434","{'address_components': [{'long_name': '1705', ...",San Jose,California,United States
1312806070,Physician: Child and Adolescent Psychiatry Pos...,Healthcare & Nursing Jobs,CompHealth,"New York, US",2019-10-25T12:39:00Z,42.522869,-75.574677,"42.522869,-75.57467700000002","{'address_components': [{'long_name': '40', 's...",Norwich,New York,United States


### Counting the columns length to see how much data is dropped during "cleaning"

In [5]:
#Maiking a copy of Geo Data
geo_csv_file_copy = geo_csv_file.copy()
geo_csv_file_copy.count()

job title       193146
category        193146
company name    193146
location        193146
date created    193146
lat             193146
lng             193146
latlng          193146
geocode_data    193146
city            193124
state           193114
country         193130
dtype: int64

#### Dropping any rows that are missing 

In [6]:
#Dropping any rows that are missing 
geo_data = geo_csv_file_copy.dropna()
geo_data.count()

job title       193114
category        193114
company name    193114
location        193114
date created    193114
lat             193114
lng             193114
latlng          193114
geocode_data    193114
city            193114
state           193114
country         193114
dtype: int64

#### Filtering only United states

In [9]:
#Calling only Rows that have United states 
dropping_non_us = geo_data[geo_data.country == 'United States']
dropping_non_us.count()

job title       192969
category        192969
company name    192969
location        192969
date created    192969
lat             192969
lng             192969
latlng          192969
geocode_data    192969
city            192969
state           192969
country         192969
dtype: int64

#### Filtering only 2019 data

In [11]:
#Calling datetime to filter only year 2019
dropping_last_year = dropping_non_us[dropping_non_us['date created'].str.contains('2019')]
dropping_last_year.count()

job title       191061
category        191061
company name    191061
location        191061
date created    191061
lat             191061
lng             191061
latlng          191061
geocode_data    191061
city            191061
state           191061
country         191061
dtype: int64

#### Dropping any duplicates based on Job ID

In [25]:
#If there are dupicates Job ID's dorp them and keep the first
clean_data = dropping_last_year.loc[~dropping_last_year.index.duplicated(keep='first')]
clean_data.count()

job title       187016
category        187016
company name    187016
location        187016
date created    187016
lat             187016
lng             187016
latlng          187016
geocode_data    187016
city            187016
state           187016
country         187016
dtype: int64

### Reviewing the how many categories are under the column category

In [26]:
#Check how many different categories you have
checking_cat = clean_data['category'].unique()
list(checking_cat)

['Healthcare & Nursing Jobs',
 'Unknown',
 'Other/General Jobs',
 'Retail Jobs',
 'Logistics & Warehouse Jobs',
 'Sales Jobs',
 'Hospitality & Catering Jobs',
 'Manufacturing Jobs',
 'Admin Jobs',
 'Accounting & Finance Jobs',
 'Customer Services Jobs',
 'Engineering Jobs',
 'IT Jobs',
 'Teaching Jobs',
 'Legal Jobs',
 'Maintenance Jobs',
 'Part time Jobs',
 'PR, Advertising & Marketing Jobs',
 'Trade & Construction Jobs',
 'Creative & Design Jobs',
 'Consultancy Jobs',
 'HR & Recruitment Jobs',
 'Property Jobs',
 'Scientific & QA Jobs',
 'Energy, Oil & Gas Jobs',
 'Domestic help & Cleaning Jobs',
 'Social work Jobs',
 'Travel Jobs',
 'Graduate Jobs',
 'Charity & Voluntary Jobs']

### Create a multiple columns to define job postings that are not defined:
* multiple columns for specific job post and category
* create an empty column called final_c which will be the new Category column
* Append all the new category information to final_c

In [28]:
#Creating columns that have unknown category, define those unknown categories and append to final_c column

#empty column to store the final Category column
clean_data['final_c'] = np.NaN

#Logistics & Warehouse Jobs
warehouse = clean_data[clean_data['job title'].str.contains("Warehous")]
clean_data['warehouse_col'] = warehouse['category'].replace('Unknown', 'Logistics & Warehouse Jobs')
clean_data['final_c'] = clean_data['final_c'].fillna(clean_data['warehouse_col'])

#Customer Services Jobs
shopper = clean_data[clean_data['job title'].str.contains("Shopp")]
clean_data['shopper_col'] = shopper['category'].replace('Unknown', 'Customer Services Jobs')
clean_data['final_c'] = clean_data['final_c'].fillna(clean_data['shopper_col'])

##Healthcare & Nursing Job##

#Physcician
physician = clean_data[clean_data['job title'].str.contains("Physician")]
clean_data['physician_col'] = physician['category'].replace('Unknown', 'Healthcare & Nursing Jobs')
clean_data['final_c'] = clean_data['final_c'].fillna(clean_data['physician_col'])

#Nurse
nurse = clean_data[clean_data['job title'].str.contains("Nurs")]
clean_data['nurse_col'] = nurse['category'].replace('Unknown', 'Healthcare & Nursing Jobs')
clean_data['final_c'] = clean_data['final_c'].fillna(clean_data['nurse_col'])

#Teaching Jobs
teach = clean_data[clean_data['job title'].str.contains("Teach")]
clean_data['teacher_col'] = teach['category'].replace('Unknown', 'Teaching Jobs')
clean_data['final_c'] = clean_data['final_c'].fillna(clean_data['teacher_col'])

#Final column Category to append into final_c column
clean_data['final_c'] = clean_data['final_c'].fillna(clean_data['category'])

clean_data

c:\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\python37\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas

,job title,category,company name,location,date created,lat,lng,latlng,geocode_data,city,state,country,final_c,warehouse_col,shopper_col,physician_col,nurse_col,teacher_col
job posting id,,,,,,,,,,,,,,,,,,
1312805843,Physician: Core Faculty Family Medicine Job in...,Healthcare & Nursing Jobs,CompHealth,"Indiana, Indiana County",2019-10-25T12:38:55Z,40.621455,-79.152535,"40.621455,-79.152535","{'address_components': [{'long_name': '1880', ...",Indiana,Pennsylvania,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1312804908,Physician: Associate Regional Medical Officer ...,Healthcare & Nursing Jobs,CompHealth,"San Jose, Santa Clara County",2019-10-25T12:38:30Z,37.390494,-121.885434,"37.390494,-121.885434","{'address_components': [{'long_name': '1705', ...",San Jose,California,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1312806070,Physician: Child and Adolescent Psychiatry Pos...,Healthcare & Nursing Jobs,CompHealth,"New York, US",2019-10-25T12:39:00Z,42.522869,-75.574677,"42.522869,-75.57467700000002","{'address_components': [{'long_name': '40', 's...",Norwich,New York,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1328242120,Physician: Anesthesiology Medical Director Job...,Healthcare & Nursing Jobs,CompHealth,"Dunkirk, Chautauqua County",2019-11-06T20:28:24Z,42.479502,-79.333932,"42.479502000000004,-79.33393199999998","{'address_components': [{'long_name': '603', '...",Dunkirk,New York,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1328242475,Physician: General Anesthesia Position North o...,Healthcare & Nursing Jobs,CompHealth,"Kittanning, Armstrong County",2019-11-06T20:29:07Z,40.816453,-79.521989,"40.816453,-79.521989","{'address_components': [{'long_name': '220', '...",Kittanning,Pennsylvania,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1211133910,Future Openings - Flow Cytometry Applications ...,Unknown,Danaher Corporation,"Cedar Rapids, Linn County",2019-07-17T16:52:16Z,41.974640,-91.659066,"41.97464,-91.65906600000001","{'address_components': [{'long_name': '804', '...",Cedar Rapids,Iowa,United States,Unknown,NaN,NaN,NaN,NaN,NaN
1306538759,"CDL Training School We Train, We Hire - Guaran...",Unknown,C.R. England,"Campbellton, Atascosa County",2019-10-21T11:40:51Z,28.721992,-98.286948,"28.721991999999997,-98.286948",{'address_components': [{'long_name': 'Interst...,Campbellton,Texas,United States,Unknown,NaN,NaN,NaN,NaN,NaN
1343686769,Sales Representative - Home Security Sales - F...,Sales Jobs,DEFENDERS,"New Milford, Bergen County",2019-11-18T18:57:20Z,40.935099,-74.019028,"40.935099,-74.019028","{'address_components': [{'long_name': '548', '...",New Milford,New Jersey,United States,Sales Jobs,NaN,NaN,NaN,NaN,NaN


### Drop the attribute 'Unknown'  from final_c column 

In [30]:
#Drop rows unknown attributes in column final_c

# Get names of indexes for which column Unknown
drop_unknown = clean_data[ clean_data['final_c'] == 'Unknown' ].index
# Delete these row indexes from dataFrame
clean_data.drop(drop_unknown , inplace=True)
clean_data

,job title,category,company name,location,date created,lat,lng,latlng,geocode_data,city,state,country,final_c,warehouse_col,shopper_col,physician_col,nurse_col,teacher_col
job posting id,,,,,,,,,,,,,,,,,,
1312805843,Physician: Core Faculty Family Medicine Job in...,Healthcare & Nursing Jobs,CompHealth,"Indiana, Indiana County",2019-10-25T12:38:55Z,40.621455,-79.152535,"40.621455,-79.152535","{'address_components': [{'long_name': '1880', ...",Indiana,Pennsylvania,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1312804908,Physician: Associate Regional Medical Officer ...,Healthcare & Nursing Jobs,CompHealth,"San Jose, Santa Clara County",2019-10-25T12:38:30Z,37.390494,-121.885434,"37.390494,-121.885434","{'address_components': [{'long_name': '1705', ...",San Jose,California,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1312806070,Physician: Child and Adolescent Psychiatry Pos...,Healthcare & Nursing Jobs,CompHealth,"New York, US",2019-10-25T12:39:00Z,42.522869,-75.574677,"42.522869,-75.57467700000002","{'address_components': [{'long_name': '40', 's...",Norwich,New York,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1328242120,Physician: Anesthesiology Medical Director Job...,Healthcare & Nursing Jobs,CompHealth,"Dunkirk, Chautauqua County",2019-11-06T20:28:24Z,42.479502,-79.333932,"42.479502000000004,-79.33393199999998","{'address_components': [{'long_name': '603', '...",Dunkirk,New York,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
1328242475,Physician: General Anesthesia Position North o...,Healthcare & Nursing Jobs,CompHealth,"Kittanning, Armstrong County",2019-11-06T20:29:07Z,40.816453,-79.521989,"40.816453,-79.521989","{'address_components': [{'long_name': '220', '...",Kittanning,Pennsylvania,United States,Healthcare & Nursing Jobs,NaN,NaN,Healthcare & Nursing Jobs,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1340503474,Online English Teacher,Teaching Jobs,Education First,"Lawrence, Douglas County",2019-11-16T03:15:34Z,39.039436,-95.203129,"39.039436,-95.203129","{'address_components': [{'long_name': '2064', ...",Lawrence,Kansas,United States,Teaching Jobs,NaN,NaN,NaN,NaN,Teaching Jobs
1317188947,"CDL Drivers: Team Up for Top Team Pay and $40,...",Logistics & Warehouse Jobs,Covenant Transport,"Henderson, Clark County",2019-10-30T00:02:22Z,35.960734,-114.939821,"35.960734,-114.939821",{'address_components': [{'long_name': 'Unnamed...,Henderson,Nevada,United States,Logistics & Warehouse Jobs,NaN,NaN,NaN,NaN,NaN
1343686769,Sales Representative - Home Security Sales - F...,Sales Jobs,DEFENDERS,"New Milford, Bergen County",2019-11-18T18:57:20Z,40.935099,-74.019028,"40.935099,-74.019028","{'address_components': [{'long_name': '548', '...",New Milford,New Jersey,United States,Sales Jobs,NaN,NaN,NaN,NaN,NaN


### Drop the temporary columns that were recently created
* Re-arrang the columns
* Rename the columns

In [16]:
#Drop unwanted columns
dropped_columns = clean_data.drop(columns=['category','warehouse_col', 'location','shopper_col','physician_col','nurse_col','teacher_col','geocode_data'])
dropped_columns.head()

,job title,company name,date created,lat,lng,latlng,city,state,country,final_c
job posting id,,,,,,,,,,
1312805843,Physician: Core Faculty Family Medicine Job in...,CompHealth,2019-10-25T12:38:55Z,40.621455,-79.152535,"40.621455,-79.152535",Indiana,Pennsylvania,United States,Healthcare & Nursing Jobs
1312804908,Physician: Associate Regional Medical Officer ...,CompHealth,2019-10-25T12:38:30Z,37.390494,-121.885434,"37.390494,-121.885434",San Jose,California,United States,Healthcare & Nursing Jobs
1312806070,Physician: Child and Adolescent Psychiatry Pos...,CompHealth,2019-10-25T12:39:00Z,42.522869,-75.574677,"42.522869,-75.57467700000002",Norwich,New York,United States,Healthcare & Nursing Jobs
1328242120,Physician: Anesthesiology Medical Director Job...,CompHealth,2019-11-06T20:28:24Z,42.479502,-79.333932,"42.479502000000004,-79.33393199999998",Dunkirk,New York,United States,Healthcare & Nursing Jobs
1328242475,Physician: General Anesthesia Position North o...,CompHealth,2019-11-06T20:29:07Z,40.816453,-79.521989,"40.816453,-79.521989",Kittanning,Pennsylvania,United States,Healthcare & Nursing Jobs


#### Re-arranging columns for preference

In [17]:
#Re-arrange columns
arrange_col_data = dropped_columns[['job title','date created','final_c','company name','city','state','country','lat','lng','latlng']]
arrange_col_data.head()

,job title,date created,final_c,company name,city,state,country,lat,lng,latlng
job posting id,,,,,,,,,,
1312805843,Physician: Core Faculty Family Medicine Job in...,2019-10-25T12:38:55Z,Healthcare & Nursing Jobs,CompHealth,Indiana,Pennsylvania,United States,40.621455,-79.152535,"40.621455,-79.152535"
1312804908,Physician: Associate Regional Medical Officer ...,2019-10-25T12:38:30Z,Healthcare & Nursing Jobs,CompHealth,San Jose,California,United States,37.390494,-121.885434,"37.390494,-121.885434"
1312806070,Physician: Child and Adolescent Psychiatry Pos...,2019-10-25T12:39:00Z,Healthcare & Nursing Jobs,CompHealth,Norwich,New York,United States,42.522869,-75.574677,"42.522869,-75.57467700000002"
1328242120,Physician: Anesthesiology Medical Director Job...,2019-11-06T20:28:24Z,Healthcare & Nursing Jobs,CompHealth,Dunkirk,New York,United States,42.479502,-79.333932,"42.479502000000004,-79.33393199999998"
1328242475,Physician: General Anesthesia Position North o...,2019-11-06T20:29:07Z,Healthcare & Nursing Jobs,CompHealth,Kittanning,Pennsylvania,United States,40.816453,-79.521989,"40.816453,-79.521989"


#### Renaming columns

In [32]:
#Renames column Names
cleansed_data = arrange_col_data.rename(columns={"final_c": "category"})
cleansed_data.head()

,job title,date created,category,company name,city,state,country,lat,lng,latlng
job posting id,,,,,,,,,,
1312805843,Physician: Core Faculty Family Medicine Job in...,2019-10-25T12:38:55Z,Healthcare & Nursing Jobs,CompHealth,Indiana,Pennsylvania,United States,40.621455,-79.152535,"40.621455,-79.152535"
1312804908,Physician: Associate Regional Medical Officer ...,2019-10-25T12:38:30Z,Healthcare & Nursing Jobs,CompHealth,San Jose,California,United States,37.390494,-121.885434,"37.390494,-121.885434"
1312806070,Physician: Child and Adolescent Psychiatry Pos...,2019-10-25T12:39:00Z,Healthcare & Nursing Jobs,CompHealth,Norwich,New York,United States,42.522869,-75.574677,"42.522869,-75.57467700000002"
1328242120,Physician: Anesthesiology Medical Director Job...,2019-11-06T20:28:24Z,Healthcare & Nursing Jobs,CompHealth,Dunkirk,New York,United States,42.479502,-79.333932,"42.479502000000004,-79.33393199999998"
1328242475,Physician: General Anesthesia Position North o...,2019-11-06T20:29:07Z,Healthcare & Nursing Jobs,CompHealth,Kittanning,Pennsylvania,United States,40.816453,-79.521989,"40.816453,-79.521989"


### Format date created column to pd.datetime

In [33]:
cleansed_data['date created'] = pd.to_datetime(cleansed_data['date created'], format = '%Y-%m-%dT%H:%M:%SZ')
cleansed_data.head(3)

,job title,date created,category,company name,city,state,country,lat,lng,latlng
job posting id,,,,,,,,,,
1312805843,Physician: Core Faculty Family Medicine Job in...,2019-10-25 12:38:55,Healthcare & Nursing Jobs,CompHealth,Indiana,Pennsylvania,United States,40.621455,-79.152535,"40.621455,-79.152535"
1312804908,Physician: Associate Regional Medical Officer ...,2019-10-25 12:38:30,Healthcare & Nursing Jobs,CompHealth,San Jose,California,United States,37.390494,-121.885434,"37.390494,-121.885434"
1312806070,Physician: Child and Adolescent Psychiatry Pos...,2019-10-25 12:39:00,Healthcare & Nursing Jobs,CompHealth,Norwich,New York,United States,42.522869,-75.574677,"42.522869,-75.57467700000002"


#### Final count of cleaned data

In [34]:
cleansed_data.count()

job title       114572
date created    114572
category        114572
company name    114572
city            114572
state           114572
country         114572
lat             114572
lng             114572
latlng          114572
dtype: int64

#### Save to a csv file. This is the csv file to be used for api city_state, visualization, and gmaps ipynb

In [35]:
#Save cleansed data to csv
cleansed_data.to_csv(cleanData)